In [15]:
from IPython.display import Markdown, Latex, Math

import sympy

# Print units using abbreviations: 'm' instead of 'meter'
sympy.printing.str.StrPrinter._default_settings['abbrev'] = True

In [16]:
class Formula:
    """
    Class that wraps lots of boilerplate.
    """
    global_values = {}
    
    @staticmethod
    def set_global_values(global_values):
        """
        Set the dictionary to use as a variable -> value registry.
        """
        Formula.global_values = global_values
    
    def __init__(self, value_name, formula=None, values=None, computed_value=None, sigfigs=6, eval_now=True):
        """
        Formula constructor.
        
        :param value_name: name of the variable, i.e. the `C` in `C = pi * d`. 
        :param formula: the formula, as either a string to pass to `sympify()` or a SymPy expression.
                        If None then the formula is assumed to just represent a variable.
        :param values: a dictionary that contains the global values state. If None, use Formula.global_values.
        :param computed_value: if the formula can't be easily computed by SymPy, use this to provide the
                               result. Useful if, for example, you can write the equation down with SymPy but
                               need numpy to compute the answer.
        :param sigfigs: How many significant figures to print the value to.
        :param eval_now: if True then compute the value of the formula now and populate `values[value_name]` with it.
        :return: 
        """
        self.value_name = value_name
        self.values = values if values is not None else Formula.global_values
  
        self.sigfigs = sigfigs
        
        if formula is None:
            # simple variable
            self.sympy_formula = sympy.sympify(value_name)
            if computed_value is None:
                assert value_name in self.values
                # computed_value = self.values[value_name]
        elif isinstance(formula, str):
            # formula defined as a string
            self.sympy_formula = sympy.sympify(formula)
        else:
            # SymPy formula
            self.sympy_formula = formula
        
        self.computed_value = computed_value

        if eval_now:
            # evaluate the formula now so subsequent formulas can just use this value
            self.values[value_name] = self.evalf()
    
    def get_sub_formula(self):
        """
        :return: a SymPy formula with substituted values.
        """
        with sympy.evaluate(False):
            return self.sympy_formula.subs(self.values)

    def evalf(self):
        """
        Evaluate this formula. If a pre-computed value was specified then that value is returned.
        Rounded to sigfigs.
        
        :return: a float value or a SymPy formula (if units are being used)
        """
        if self.computed_value is not None:
            return self.computed_value
        
        with sympy.evaluate(True):
            return self.get_sub_formula().evalf(self.sigfigs, chop=True)
        
    def _get_aligned_latex_str(self):
        """
        Private method that returns a Latex `align` version of this formula.
        """
        # Output the result as an aligned equation.
        # The asterisk in `\begin{align*}` tells Latex to not add an equation number on the right hand side.
        # Note that str.format() uses curly braces for formatting, so any curly braces that are not part
        # of a string replacement must be doubled. Hence `\begin{{align*}}` instead of `\begin{align*}`.
        fmt = r"""
        \begin{{align*}}
        {value_name} &= """
        
        if str(self.sympy_formula) != self.value_name:
            # this has a formula (not just a simple variable, for example)
            fmt += r"""{formula} \\
            &= """
            if self.computed_value is None:
                # not using a precomputed value, we can show variable values substituted in the formula
                fmt += r"""{formula_sub} \\
            &= """
        
        # the value
        fmt += r"{value}"
        
        # close the env
        fmt += r"""
        \end{{align*}}"""
        
        return fmt.format(value_name=self.value_name,
                   formula=sympy.latex(self.sympy_formula),
                   formula_sub=sympy.latex(self.get_sub_formula()),
                   value=sympy.latex(self.evalf()))
    
    def _get_single_line_latex_str(self):
        """
        Private method that returns a Latex one liner version of this formula.
        """
        fmt = "{value_name}"
        if str(self.sympy_formula) != self.value_name:
            # this has a formula (not just a simple variable, for example)
            fmt += " = {formula}"
            if self.computed_value is None:
                # not using a precomputed value, we can show variable values substituted in the formula
                fmt += " = {formula_sub}"
        
        # the value
        fmt += " = {value}"
            
        return fmt.format(
            value_name=self.value_name,
            formula=sympy.latex(self.sympy_formula),
            formula_sub=sympy.latex(self.get_sub_formula()),
            value=sympy.latex(self.evalf()))
    
    def get_display(self, oneline=True):
        """
        Get a Jupyter-friendly object that can be used to emit this formula on its own line.
        
        This includes the variable name, the formula itself, formula substituted with values,
        and the final computed value. Some parts may be omitted as appropriate.
        
        :param online: if True then then all parts are on one line, if False then a
                       Latex align environment is used where each part is on its own line.
        """
        if oneline:
            return Markdown("$$ {latex} $$".format(latex=self._get_single_line_latex_str()))
        else:
            return Latex(self._get_aligned_latex_str())
        
    def get_inline(self):
        """
        Get a Jupyter-friendly object that can be used to emit this formula inside normal text.
        
        This includes the variable name, the formula itself, formula substituted with values,
        and the final computed value. Some parts may be omitted as appropriate.
        """
        return Math(self._get_single_line_latex_str())
    
    def get_inlinev(self):
        """
        Get a Jupyter-friendly object that only contains the value, nothing else.
        
        This is a pretty-printed version of `evalf()`.
        """
        return Math(sympy.latex(self.evalf()))
    
    def get_inlinev_latex(self):
        """
        Same as get_inlinev(), but emitted as a Latex-formatted string, wrapped with `$`.
        
        Useful when this is necessary for things like including the value inside a Markdown table
        that must be constructed as a string.
        """
        return "$" + sympy.latex(self.evalf()) + "$"
                     